Materialization for metric views is an [experimental](https://docs.databricks.com/aws/en/release-notes/release-types) feature. The idea is that you can create a materialized view on top of a metric view definition, and that will improve query performance.  

Check out more details about it here: https://docs.databricks.com/aws/en/metric-views/materialization

In [0]:
# Define parameters
dbutils.widgets.text("CATALOG_NAME", "pedroz_catalog", "Catalog Name")
dbutils.widgets.text("SCHEMA_NAME", "metric_views_schema", "Schema Name")
CATALOG_NAME = dbutils.widgets.get("CATALOG_NAME")
SCHEMA_NAME = dbutils.widgets.get("SCHEMA_NAME")

In [0]:
%sql

CREATE OR REPLACE VIEW ${CATALOG_NAME}.${SCHEMA_NAME}.materialized_sales_aggregations WITH METRICS LANGUAGE YAML AS 
$$
version: 0.1

source: ${CATALOG_NAME}.${SCHEMA_NAME}.fact_sales_denormalized

dimensions:
  - name: sales_date
    expr: sales_date
  - name: store_type
    expr: store_type
  - name: product_category
    expr: category
  - name: campaign_name
    expr: campaign_name

measures:
  - name: sum_sales
    expr: SUM(total_amount)

  - name: avg_sales
    expr: AVG(total_amount)

materialization:
  schedule: every 1 hours
  mode: relaxed

  materialized_views:
    - name: baseline
      type: unaggregated

    - name: product_agg
      type: aggregated
      dimensions:
        - product_category
      measures:
        - sum_sales
        - avg_sales

    - name: store_agg
      type: aggregated
      dimensions:
        - store_type
      measures:
        - sum_sales
        - avg_sales
$$

In [0]:
%sql
DESCRIBE EXTENDED ${CATALOG_NAME}.${SCHEMA_NAME}.materialized_sales_aggregations

In [0]:
# Queries like the one below should run much faster, as the query engine will understand that it can use the materialized results!

# To check if a query is using a materialized view, run EXPLAIN EXTENDED on your query to see the query plan. If the query is using materialized views, the leaf node includes __materialization_mat___metric_view and the name of the materialization from the YAML file.

In [0]:
%sql
EXPLAIN EXTENDED
SELECT 
  product_category, 
  MEASURE(`sum_sales`) AS `sum_sales`, 
  MEASURE(`avg_sales`) AS `avg_sales`
FROM pedroz_catalog.metric_views_schema.materialized_sales_aggregations
GROUP BY product_category